# Dash Callbacks

Una vez que hemos revisado los componentes HTML y los componentes CSS comenzaremos a trabajar con el diseño de los dashboards. Por medio de llamadas *callbacks* se hará la interaccón y conexión con estos componentes. Los pasos para crear un *callback* son los siguientes:
* Crear una función que regrese una salida esperada
* Utililizar un decorador en la función creada
    * Establecer una salida al ID del componente
    * Establecer un entrada al ID del componente
* Conectar las propiedades deseadas

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output  # permite crear los callbacks

In [2]:
app = JupyterDash('CallbackExample')

app.layout = html.Div([
                dcc.Input(id = 'my-id', value = 'Initial Text', type = 'text'),
                html.Div(id = 'my-div')
            ])

@app.callback(Output(component_id = 'my-div', component_property = 'children'),
            [Input(component_id = 'my-id', component_property = 'value')]
            )
def update_output_div(input_value):
    return "You entered: {}".format(input_value)

In [3]:
app #nota el botón undo!!

## Dash Callbacks para gráficos

In [27]:
from jupyter_plotly_dash import JupyterDash

import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [28]:
mainpath = "/home/oscar/Documentos/udemy/plotly-dash-joseport/Plotly-Dashboards-with-Dash-master/"
filename = "Data/gapminderDataFiveYear.csv"
fullpath = mainpath + filename

data = pd.read_csv(fullpath)

In [29]:
data.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [30]:
app = JupyterDash('DashCallbackExample')


year_options = []
for year in data['year'].unique():
    year_options.append({'label': str(year), 'value':year})

app.layout = html.Div([
                    dcc.Graph(id = 'graph'),
                    dcc.Dropdown(id = 'year-picker', options = year_options,
                                 value = data['year'].min())
                    ])

#se puede omitir el nombre de los parámetros y especificarloss en orden
@app.callback(Output(component_id = 'graph', component_property = 'figure'), 
                     [Input('year-picker', 'value')])
def update_figure(selected_year):
    
    # data only for selected year from dropdown
    filtered_data = data[data['year'] == selected_year]
    
    traces = []
    
    for continent_name in filtered_data['continent'].unique():
        data_by_continent = filtered_data[filtered_data['continent']==continent_name]
        traces.append(go.Scatter(
            x = data_by_continent['gdpPercap'],
            y = data_by_continent['lifeExp'],
            mode = 'markers',
            opacity = 0.7,
            marker = {'size': 10},
            name = continent_name
                     ))
        
    return {'data': traces,
           'layout':go.Layout(title = 'My Plot',
                             xaxis = {'title': 'GDP per Cap', 'type':'log'},
                             yaxis = {'title': 'Life Expectancy'} 
                             )}

In [31]:
app

## Multiple inputs

Hemos visto una salida gráfica para una entrada de texto. Ahora veremos la manera de implementar un Dashboard que capture varias entradas y tenga una sola salida en un mismo gráfico.

In [32]:
from jupyter_plotly_dash import JupyterDash

import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [38]:
mainpath = "/home/oscar/Documentos/udemy/plotly-dash-joseport/Plotly-Dashboards-with-Dash-master/"
filename = "Data/mpg.csv"
fullpath = mainpath + filename

In [39]:
data = pd.read_csv(fullpath)

In [40]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [41]:
app = JupyterDash("DashCallbackMultInputs")

#['mpg', 'hp', 'displacement']
features = data.columns

app.layout = html.Div([
                html.Div([
                    dcc.Dropdown(id = 'xaxis',
                                options = [{'label': i, 'value': i} for i in features], # crea options con comprehlist
                                value = 'displacement')
                ], style = {'width': '48%', 'display': 'inline-block'}),
                html.Div([
                    dcc.Dropdown(id ='yaxis',
                                options = [{'label': i, 'value': i} for i in features],
                                value = 'mpg')
                ], style = {'width': '48%', 'display': 'inline-block'}),
                dcc.Graph(id='feature-graphic')
], style={'pading':10})


@app.callback(Output('feature-graphic', 'figure'), #component_id, component_property
             [Input('xaxis', 'value'),
              Input('yaxis', 'value')
             ])
def update_graph(xaxis_name, yaxis_name):
    return {'data':[go.Scatter(x = data[xaxis_name],
                               y = data[yaxis_name],
                               text = data['name'],
                               mode = 'markers',
                               marker = {'size':10,
                                         'opacity': 0.5,
                                         'line': {'width':0.5, 'color':'white'}
                                        },
                               )],
            'layout': go.Layout(title = 'My Dashboard for MPG',
                                xaxis = {'title':xaxis_name},
                                yaxis = {'title':yaxis_name},
                                hovermode = 'closest')
           }

In [42]:
app

## Multiple Outputs

Anteriormente se vio la manera de crear multiples entradas conectadas al dashboard. Para obtener multiples salidas será necesario crear multiples pares function/callbacks-decorators en el script. Veremos un ejemplo de un dashboard que proporciona una de múltiples salidas.

In [21]:
from jupyter_plotly_dash import JupyterDash

import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

import base64  # despliega imágenes en un html

In [22]:
mainpath = "/home/oscar/Documentos/udemy/plotly-dash-joseport/Plotly-Dashboards-with-Dash-master/"
filename = "Data/wheels.csv"
fullpath = mainpath + filename

data = pd.read_csv(fullpath)

In [23]:
data.head()

,wheels,color,image
0,1,red,redunicycle.jpg
1,1,blue,blueunicycle.jpg
2,1,yellow,yellowunicycle.jpg
3,2,red,redbicycle.jpg
4,2,blue,bluemotorcycle.jpg


In [24]:
app = JupyterDash('DashCallbackMultOutputs')

def encode_image(image_file):
    encoded = base64.b64encode(open(image_file, 'rb').read())
    return 'data:image/png;base64, {}'.format(encoded.decode())


app.layout = html.Div([
                dcc.RadioItems(id = 'wheels',
                               options = [{'label': i, 'value': i} for i in data['wheels'].unique()],
                               value = 1
                               ),
                html.Div(id = 'wheels-output'),
                html.Hr(), #pinta una línea
                dcc.RadioItems(id = 'colors',
                                options = [{'label': i, 'value': i} for i in data['color'].unique()],
                                value = 'blue'
                              ),
                html.Div(id = 'colors-output'),
                html.Img(id = 'display-image', src='children', height = 300)
], style={'fontFamily':'helvetica', 'fontSize': 18 })

@app.callback(Output('wheels-output', 'children'),
              [Input('wheels', 'value')])
def callback_a(wheels_value):
    return "you chose {}".format(wheels_value)


@app.callback(Output('colors-output', 'children'),
              [Input('colors', 'value')])
def callback_b(color_value):
    return "you chose {}".format(color_value)

@app.callback(Output('display-image', 'src'),
              [Input('wheels', 'value'),
               Input('colors', 'value')])
def callback_images(wheel, color):
    file = data[(data['wheels'] == wheel) & (data['color'] == color)]['image'].values[0]
    mainpath = "/home/oscar/Documentos/udemy/plotly-dash-joseport/Plotly-Dashboards-with-Dash-master/"
    filename = "Data/Images/" + file
    return encode_image(mainpath + filename)

In [25]:
app

### Ejercicio Interactive Components

**Objetivo:** Crear un Dashboard que tome dos o mas valores de entrada y regrese el producto de ambos como salida

In [43]:
# perform inputs here

from jupyter_plotly_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [44]:
# launch the application
app = JupyterDash('InteractiveComponentsExercise')

In [45]:
# create a Dash layout that contains input components
# and at least one output. Assign ID's to each component

app.layout = html.Div([
                dcc.RangeSlider(id = 'range-slider',
                                min = -10,
                                max = 10,
                                marks = {i:str(i) for i in range(-10, 11)},
                                value = [-1,1]),
                html.H1(id = 'product')
])

@app.callback(Output('product', 'children'),
             [Input('range-slider', 'value')])
def update_value(value_list):
    return value_list[0]*value_list[1]

In [46]:
app

In [53]:
i for i in range(-10, 11):
    print(i)

-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
